数据集处理：


    原数据集V1:(3d)

In [1]:
import numpy as np

# 加载数据
test_joint = np.load('data/test_A_joint.npy')  # 形状 (2000, 3, 300, 17, 2)
test_label = np.load('data/test_A_label.npy')  # 形状 (2000,)
train_joint = np.load('data/train_joint.npy')  # 形状 (16432, 3, 300, 17, 2)
train_label = np.load('data/train_label.npy')  # 形状 (16432,)

# train_reshaped = train_joint[:, 1:, :, :, :]
# print(train_reshaped.shape)

# test_reshaped = test_joint[:, 1:, :, :, :]
# print(test_reshaped.shape)

test_joint = np.transpose(test_joint, (0,4,2,3,1))
train_joint = np.transpose(train_joint, (0,4,2,3,1))

# # 调整维度顺序：将 (2000, 3, 300, 17, 2) 变为 (2000, 2, 300, 17, 3)
# test_reshaped = np.transpose(test_reshaped, (0, 2, 4, 3, 1))  # 形状变为 (2000, 2, 300, 17, 3)
# train_reshaped = np.transpose(train_reshaped, (0, 2, 4, 3, 1))  # 形状变为 (16432, 2, 300, 17, 3)

# y_test 和 y_train 保持不变
y_test = test_label  # 形状 (2000,)
y_train = train_label  # 形状 (16432,)

# 保存为 .npz 文件
np.savez('V1_3d_4_J.npz', x_train=train_joint, y_train=y_train, x_test=test_joint, y_test=y_test)

print("V1.npz 文件已保存")


V1.npz 文件已保存


原数据集V1：(2d)

In [5]:
import numpy as np

# 加载数据
test_joint = np.load('data/test_A_joint.npy')  # 形状 (2000, 3, 300, 17, 2)
test_label = np.load('data/test_A_label.npy')  # 形状 (2000,)
train_joint = np.load('data/train_joint.npy')  # 形状 (16432, 3, 300, 17, 2)
train_label = np.load('data/train_label.npy')  # 形状 (16432,)

train_reshaped = train_joint[:, 1:, :, :, :]
print(train_reshaped.shape)

test_reshaped = test_joint[:, 1:, :, :, :]
print(test_reshaped.shape)

# test_joint = np.transpose(test_joint, (0,4,2,3,1))
# train_joint = np.transpose(train_joint, (0,4,2,3,1))

# 调整维度顺序：将 (2000, 3, 300, 17, 2) 变为 (2000, 2, 300, 17, 3)
test_reshaped = np.transpose(test_reshaped, (0, 2, 4, 3, 1))  # 形状变为 (2000, 300, 2, 17, 2)
train_reshaped = np.transpose(train_reshaped, (0, 2, 4, 3, 1))  # 形状变为 (16432, 300, 2, 17, 2)

# y_test 和 y_train 保持不变
y_test = test_label  # 形状 (2000,)
y_train = train_label  # 形状 (16432,)

# 保存为 .npz 文件
np.savez('V1_2d_4_J.npz', x_train=train_reshaped, y_train=y_train, x_test=test_reshaped, y_test=y_test)

print("V1.npz 文件已保存")


(16432, 2, 300, 17, 2)
(2000, 2, 300, 17, 2)
V1.npz 文件已保存


tegcn使用类别：
numpy版本会影响读取，使用相同numpy版本产生pkl与运行tegcn (python=3.8.12 numpy==1.22.3)

In [ ]:
import numpy as np
import pickle

# 读取 .npy 文件
data = np.load('data/train_label.npy')  # 假设文件名为 data.npy
print(data.shape)

# 创建新的形状为 (2, 2000) 的数组
new_data = np.empty((2, 16432), dtype=data.dtype)
new_data[1] = data  # 将原数组赋值给第二个维度

# new_data[0] = ['test_' + str(i) for i in range(16432)]

# 保存为 .pkl 文件
with open('train_label.pkl', 'wb') as file:
    pickle.dump(new_data, file)

print("转换完成，已保存为 train_label.pkl")


# 读取 .npy 文件
data = np.load('data/test_A_label.npy')  # 假设文件名为 data.npy
print(data.shape)

# 创建新的形状为 (2, 2000) 的数组
new_data = np.empty((2, 2000), dtype=data.dtype)
new_data[1] = data  # 将原数组赋值给第二个维度

# new_data[0] = ['test_' + str(i) for i in range(16432)]

# 保存为 .pkl 文件
with open('test_label.pkl', 'wb') as file:
    pickle.dump(new_data, file)

print("转换完成，已保存为 test_label.pkl")


: 

数据集扩充

In [7]:
import numpy as np


# label对称组
index = [(3, 4), (6, 7), (11, 12), (13, 14), (15, 16), (34, 35), (38, 39), (49, 50), (51, 52), (84, 85),
         (102, 103), (104, 105), (106, 107), (108, 109), (112, 113), (119, 120), (144, 145), (153, 154)]

# 时间帧倒放
def reverse_frame(data_file, label_file):
    dataset = np.load(data_file)
    labels = np.load(label_file)
    length = dataset.shape[0]
    count = 0
    for data, label in zip(dataset, labels):
        for label1, label2 in index:    # 对称label
            if label == label1:
                sample = np.zeros((3, 300, 17, 2))
                zero_start = find_zero_start(data, axis=1)
                sample[:, :zero_start, :, :] = data[:, zero_start-1::-1, :, :]  # 仅倒放非零段
                dataset = np.append(dataset, [sample], axis=0)
                labels = np.append(labels, label2)
            elif label == label2:
                sample = np.zeros((3, 300, 17, 2))
                zero_start = find_zero_start(data, axis=1)
                sample[:, :zero_start, :, :] = data[:, zero_start-1::-1, :, :]  # 仅倒放非零段
                dataset = np.append(dataset, [sample], axis=0)
                labels = np.append(labels, label1)
        count += 1
        if(count % 1000 == 0 or count == length):
            print(f"已处理：{(count/length)*100:.2f}%")
    return dataset, labels

# 截取非零段
def find_zero_start(arr, axis=1):
    shape = arr.shape
    for i in range(shape[axis]):
        if np.sum(arr[:, i, :, :]) == 0:
            return i
    return shape[axis]


# if __name__ == '__main__':
#     sets = ['bone', 'joint', 'bone_motion', 'joint_motion'] #所需模态
#     for set in sets:
#         data_file = f"./data/train_{set}.npy"  # 训练集data路径
#         label_file = "./data/train_label.npy"  # 训练集label路径
#         data_new, label_new = reverse_frame(data_file, label_file)
#         print(data_new.shape)

#         # 3D
#         data_new = np.transpose(data_new, (0, 4, 2, 3, 1))
#         np.savez(f"./data_new/train_{set}.npz", x_train=data_new, y_train=label_new)

#         # # 2D
#         # data_new = np.transpose(data_new, (0, 2, 4, 3, 1))
#         # np.savez(f"../data_new/train_{set}_2d.npz", x_train=data_new, y_train=label_new)

if __name__ == '__main__':
    sets = ['joint'] #所需模态
    for set in sets:
        data_file = f"./data/train_{set}.npy"  # 训练集data路径
        label_file = "./data/train_label.npy"  # 训练集label路径
        data_new, label_new = reverse_frame(data_file, label_file)
        print(data_new.shape)

        # 3D
        data_new = np.transpose(data_new, (0, 4, 2, 3, 1))
        np.savez(f"./data_new/train_{set}.npz", x_train=data_new, y_train=label_new)

        # # 2D
        # data_new = np.transpose(data_new, (0, 2, 4, 3, 1))
        # np.savez(f"../data_new/train_{set}_2d.npz", x_train=data_new, y_train=label_new)

已处理：6.09%
已处理：12.17%
已处理：18.26%
已处理：24.34%
已处理：30.43%
已处理：36.51%
已处理：42.60%
已处理：48.69%
已处理：54.77%
已处理：60.86%
已处理：66.94%
已处理：73.03%
已处理：79.11%
已处理：85.20%
已处理：91.29%
已处理：97.37%
已处理：100.00%
(20248, 3, 300, 17, 2)


B数据集处理

In [1]:
import numpy as np

test_joint = np.load('data/test_joint_B.npy')  # 形状 (4599, 3, 300, 17, 2)

print(test_joint.shape)
test_joint = np.transpose(test_joint, (0, 4, 2, 3, 1))
print(test_joint.shape)

test_label = np.zeros(4599, )
print(test_label.shape)

np.savez('V1_test_B.npz', x_test=test_joint, y_test=test_label)
print("已保存")

np.save('data/test_label_B.npy', test_label)



(4599, 3, 300, 17, 2)
(4599, 2, 300, 17, 3)
(4599,)
已保存


In [2]:
import numpy as np

test_joint = np.load('data/test_joint_B.npy')  # 形状 (4599, 3, 300, 17, 2)

print(test_joint.shape)

print(test_joint.shape)

test_label = np.zeros(4599, )
print(test_label.shape)


test_reshaped = test_joint[:, 1:, :, :, :]
print(test_reshaped.shape)
test_reshaped = np.transpose(test_reshaped, (0, 2, 4,3,1))
print(test_reshaped.shape)

np.savez('V1_test_B_2d.npz', x_test=test_reshaped, y_test=test_label)
print("已保存")

# np.save('data/test_label_B.npy', test_label)

(4599, 3, 300, 17, 2)
(4599, 3, 300, 17, 2)
(4599,)
(4599, 2, 300, 17, 2)
(4599, 300, 2, 17, 2)
已保存
